In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.9 MB/s eta 0:00:00


In [3]:
import openai
import json
import requests

openai.api_key = "Hier API-Key einfügen"

feiertage_api_url = "https://feiertage-api.de/api/"

prompt = "Wir haben den 3.7.2023, wann ist der nächste gesetzliche Feiertag in Hessen?"

messages = [{"role": "user", "content": prompt}]

def feiertage(jahr):
  print(f"Hole Feiertage für Jahr {jahr}")
  print(feiertage_api_url + "?jahr=" + jahr)
  return requests.get(feiertage_api_url + "?jahr=" + jahr).json()


functions = [
  {
      "name": "feiertage",
      "description": "Ermittelt die gesetzlichen Feiertage in Deutschland",
      "parameters": {
          "type": "object",
          "properties": {
              "jahr": {"type": "string", "description": "Das Jahr in dem die Feiertage liegen sollen, Beispiel '2023'"},
          },
          "required": ["jahr"],
      },
  }
]



def execute_function_call(message):
    if message["function_call"]["name"] == "feiertage":
        jahr = json.loads(message["function_call"]["arguments"])["jahr"]
        results = feiertage(jahr)
    else:
        results = f"Fehler! Es existiert keine Funktion mit dem Namen: {message['function_call']['name']}"
    return results





response = openai.ChatCompletion.create(
	model="gpt-3.5-turbo",
	messages=messages,
  functions = functions
)

print(response)

ai_msg = response["choices"][0]["message"]
messages.append(ai_msg)

if ai_msg.get("function_call"):
  results = execute_function_call(ai_msg)
  print(results)
  messages.append({"role": "user", "content": ""})
  response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=messages,functions = functions)
  print(response)


{
  "id": "chatcmpl-7YqFEqFINsLnzX2GJa3uYeGo6UNNO",
  "object": "chat.completion",
  "created": 1688538336,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "feiertage",
          "arguments": "{\n  \"jahr\": \"2023\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 99,
    "completion_tokens": 18,
    "total_tokens": 117
  }
}
Hole Feiertage für Jahr 2023
https://feiertage-api.de/api/?jahr=2023
{'BW': {'Neujahrstag': {'datum': '2023-01-01', 'hinweis': ''}, 'Heilige Drei Könige': {'datum': '2023-01-06', 'hinweis': ''}, 'Gründonnerstag': {'datum': '2023-04-06', 'hinweis': 'Gemäß § 4 Abs. 3 des Feiertagsgesetzes von Baden-Württemberg[10] haben Schüler am Gründonnerstag und am Reformationstag schulfrei. In der Regel legt das Kultusministerium die Ferientermine so fest, dass diese beiden Tage i